In [ ]:
!apt-get update
!apt-get install build-essential
!apt-get install -y libcudnn8 libcudnn8-dev
!apt-get install g++

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [46.8 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,467 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/c2d4u.te

In [ ]:
%%writefile matmul.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <math.h>
#include <stdio.h>
#include <time.h>
#include <iostream>
#include <stdlib.h>
#include "GpuTimer.h"
using namespace std;

#define BLOCK_SIZE 16
#define TILE_WIDTH 16

void matMulCPU(float* A, float* B, float* C, int numARows, int numACols, int numBCols) {
    int i, j, k;
    int offsetA, offsetB;
    float cumSum;

    for (i = 0; i < numARows; i++) {
        for (j = 0; j < numBCols; j++) {
            cumSum = 0;
            for (k = 0; k < numACols; k++) {
                // linearize index
                offsetA = i*numACols + k;
                offsetB = k*numBCols + j;

                // accumulate element-wise product
                cumSum += A[offsetA] * B[offsetB];
            }
            C[i*numBCols + j] = cumSum;
        }
    }
}

__global__ void matMulGPU(float* A, float* B, float* C, int numARows, int numACols, int numBCols) {
    // allocate shared memory
    __shared__ float sharedA[TILE_WIDTH][TILE_WIDTH];
    __shared__ float sharedB[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x; int by = blockIdx.y;
    int tx = threadIdx.x; int ty = threadIdx.y;

    // coordinates for C
    int row = by * TILE_WIDTH + ty;
    int col = bx * TILE_WIDTH + tx;

    float cumSum = 0;
    for (int m = 0; m < ceil(numACols/(float)TILE_WIDTH); m++) {
        // load tiles
        if ((row < numARows) && ((m*TILE_WIDTH + tx) < numACols))
            sharedA[ty][tx] = A[row*numACols + m*TILE_WIDTH + tx];
        else
            sharedA[ty][tx] = 0;
        if ((col < numBCols) && ((m*TILE_WIDTH + ty) < numACols))
            sharedB[ty][tx] = B[(m*TILE_WIDTH + ty)*numBCols + col];
        else
            sharedB[ty][tx] = 0;
        // pause until all threads have loaded tile values
        __syncthreads();

        // compute partial dot product (for individual thread)
        for (int k = 0; k < TILE_WIDTH; k++) {
            cumSum += sharedA[ty][k] * sharedB[k][tx];
        }
        // wait until all threads have used tile values
        __syncthreads();
    }
    if((row < numACols) && (col < numBCols)) {
        C[row*numBCols + col] = cumSum;
    }
}


int main(int argc, char* argv[]) {
    int numARows = 960;
    int numACols = 640;
    int numBCols = 800;

    if (argc == 4) {
        numARows = std::stoi(argv[1]);
        numACols = std::stoi(argv[2]);
        numBCols = std::stoi(argv[3]);
    }

    cout<<"Size of matrix A: "<<numARows<<" "<<numACols<<endl;
    cout<<"Size of matrix B: "<<numACols<<" "<<numBCols<<endl;

    // timers
    GpuTimer timer0, timer1, timer2, timer3;

    size_t sizeA = numARows * numACols * sizeof(float);
    size_t sizeB = numACols * numBCols * sizeof(float);
    size_t sizeC = numARows * numBCols * sizeof(float);

    // allocate host memory
    float* h_A = (float*)malloc(sizeA);
    float* h_B = (float*)malloc(sizeB);
    float* h_C = (float*)malloc(sizeC);
    float* h_C_CPU = (float*)malloc(sizeC);

    // initialize host matrices
    int i, j, offset;
    for (i = 0; i <  numARows; i++) {
        for (j = 0; j < numACols; j++) {
            offset = i*numACols + j;
            h_A[offset] = sin(i);
        }
    }
    for (i = 0; i <  numACols; i++) {
        for (j = 0; j < numBCols; j++) {
            offset = i*numBCols + j;
            h_B[offset] = cos(j);
        }
    }

    // allocate device matrices
    float* d_A;
    float* d_B;
    float* d_C;
    timer0.Start();
    cudaMalloc((void**)&d_A, sizeA);
    cudaMalloc((void**)&d_B, sizeB);
    cudaMalloc((void**)&d_C, sizeC);
    timer0.Stop();
    printf("Time to allocate memory on the device is: %f msecs.\n", timer0.Elapsed());

    // transfer to GPU
    timer1.Start();
    cudaMemcpy(d_A, h_A, sizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, sizeB, cudaMemcpyHostToDevice);
    timer1.Stop();
    printf("Time to copy the Matrix from host to device is: %f msecs.\n", timer1.Elapsed());

    // kernel launch
    dim3 threadPerBlock(BLOCK_SIZE, BLOCK_SIZE, 1);
    dim3 blockPerGrid(ceil(numBCols/(float)BLOCK_SIZE), ceil(numACols/(float)BLOCK_SIZE), 1);
    timer2.Start();
    matMulGPU<<<blockPerGrid, threadPerBlock>>>(d_A, d_B, d_C, numARows, numACols, numBCols);
    timer2.Stop();
    printf("Implemented CUDA code ran in: %f msecs.\n", timer2.Elapsed());

    // transfer to CPU
    timer3.Start();
    cudaMemcpy(h_C, d_C, sizeC, cudaMemcpyDeviceToHost);
    timer3.Stop();
    printf("Time to copy the resulting Matrix from device to host is: %f msecs.\n", timer3.Elapsed());

    clock_t begin = clock();
    matMulCPU(h_A, h_B, h_C_CPU, numARows, numACols, numBCols);
    clock_t end = clock();
    double time_spent = (double)(end - begin) / CLOCKS_PER_SEC * 1000;
    printf("Implemented CPU code ran in: %f msecs.\n", time_spent);

    free(h_A); free(h_B); free(h_C); free(h_C_CPU);
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);

    return 0;
}

Overwriting matmul.cu


In [ ]:

%%writefile matmulTiled.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <math.h>
#include <stdio.h>
#include <time.h>
#include <iostream>
#include <stdlib.h>
#include "GpuTimer.h"
using namespace std;

#define BLOCK_SIZE 16
#define TILE_WIDTH 16

void matMulCPU(float* A, float* B, float* C, int numARows, int numACols, int numBCols) {
    int i, j, k;
    int offsetA, offsetB;
    float cumSum;

    for (i = 0; i < numARows; i++) {
        for (j = 0; j < numBCols; j++) {
            cumSum = 0;
            for (k = 0; k < numACols; k++) {
                // linearize index
                offsetA = i*numACols + k;
                offsetB = k*numBCols + j;

                // accumulate element-wise product
                cumSum += A[offsetA] * B[offsetB];
            }
            C[i*numBCols + j] = cumSum;
        }
    }
}

__global__ void matMulGPU(float* A, float* B, float* C, int numARows, int numACols, int numBCols) {
    // allocate shared memory
    __shared__ float sharedA[TILE_WIDTH][TILE_WIDTH];
    __shared__ float sharedB[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x; int by = blockIdx.y;
    int tx = threadIdx.x; int ty = threadIdx.y;

    // coordinates for C
    int row = by * TILE_WIDTH + ty;
    int col = bx * TILE_WIDTH + tx;

    float cumSum = 0;
    for (int m = 0; m < ceil(numACols/(float)TILE_WIDTH); m++) {
        // load tiles
        if ((row < numARows) && ((m*TILE_WIDTH + tx) < numACols))
            sharedA[ty][tx] = A[row*numACols + m*TILE_WIDTH + tx];
        else
            sharedA[ty][tx] = 0;
        if ((col < numBCols) && ((m*TILE_WIDTH + ty) < numACols))
            sharedB[ty][tx] = B[(m*TILE_WIDTH + ty)*numBCols + col];
        else
            sharedB[ty][tx] = 0;
        // pause until all threads have loaded tile values
        __syncthreads();

        // compute partial dot product (for individual thread)
        for (int k = 0; k < TILE_WIDTH; k++) {
            cumSum += sharedA[ty][k] * sharedB[k][tx];
        }
        // wait until all threads have used tile values
        __syncthreads();
    }
    if((row < numACols) && (col < numBCols)) {
        C[row*numBCols + col] = cumSum;
    }
}


int main(int argc, char* argv[]) {
    int numARows = 960;
    int numACols = 640;
    int numBCols = 800;

    if (argc == 4) {
        numARows = std::stoi(argv[1]);
        numACols = std::stoi(argv[2]);
        numBCols = std::stoi(argv[3]);
    }

    cout<<"Size of matrix A: "<<numARows<<" "<<numACols<<endl;
    cout<<"Size of matrix B: "<<numACols<<" "<<numBCols<<endl;

    // timers
    GpuTimer timer0, timer1, timer2, timer3;

    size_t sizeA = numARows * numACols * sizeof(float);
    size_t sizeB = numACols * numBCols * sizeof(float);
    size_t sizeC = numARows * numBCols * sizeof(float);

    // allocate host memory
    float* h_A = (float*)malloc(sizeA);
    float* h_B = (float*)malloc(sizeB);
    float* h_C = (float*)malloc(sizeC);
    float* h_C_CPU = (float*)malloc(sizeC);

    // initialize host matrices
    int i, j, offset;
    for (i = 0; i <  numARows; i++) {
        for (j = 0; j < numACols; j++) {
            offset = i*numACols + j;
            h_A[offset] = sin(i);
        }
    }
    for (i = 0; i <  numACols; i++) {
        for (j = 0; j < numBCols; j++) {
            offset = i*numBCols + j;
            h_B[offset] = cos(j);
        }
    }

    // allocate device matrices
    float* d_A;
    float* d_B;
    float* d_C;
    timer0.Start();
    cudaMalloc((void**)&d_A, sizeA);
    cudaMalloc((void**)&d_B, sizeB);
    cudaMalloc((void**)&d_C, sizeC);
    timer0.Stop();
    printf("Time to allocate memory on the device is: %f msecs.\n", timer0.Elapsed());

    // transfer to GPU
    timer1.Start();
    cudaMemcpy(d_A, h_A, sizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, sizeB, cudaMemcpyHostToDevice);
    timer1.Stop();
    printf("Time to copy the Matrix from host to device is: %f msecs.\n", timer1.Elapsed());

    // kernel launch
    dim3 threadPerBlock(BLOCK_SIZE, BLOCK_SIZE, 1);
    dim3 blockPerGrid(ceil(numBCols/(float)BLOCK_SIZE), ceil(numACols/(float)BLOCK_SIZE), 1);
    timer2.Start();
    matMulGPU<<<blockPerGrid, threadPerBlock>>>(d_A, d_B, d_C, numARows, numACols, numBCols);
    timer2.Stop();
    printf("Implemented CUDA code ran in: %f msecs.\n", timer2.Elapsed());

    // transfer to CPU
    timer3.Start();
    cudaMemcpy(h_C, d_C, sizeC, cudaMemcpyDeviceToHost);
    timer3.Stop();
    printf("Time to copy the resulting Matrix from device to host is: %f msecs.\n", timer3.Elapsed());

    clock_t begin = clock();
    matMulCPU(h_A, h_B, h_C_CPU, numARows, numACols, numBCols);
    clock_t end = clock();
    double time_spent = (double)(end - begin) / CLOCKS_PER_SEC * 1000;
    printf("Implemented CPU code ran in: %f msecs.\n", time_spent);

    free(h_A); free(h_B); free(h_C); free(h_C_CPU);
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);

    return 0;
}

Overwriting matmulTiled.cu


In [ ]:

# Archivo para tomar tiempos
%%writefile GpuTimer.h
#ifndef __GPU_TIMER_H__
#define __GPU_TIMER_H__

struct GpuTimer{
      cudaEvent_t start;
      cudaEvent_t stop;

      GpuTimer(){
            cudaEventCreate(&start);
            cudaEventCreate(&stop);
      }

      ~GpuTimer(){
            cudaEventDestroy(start);
            cudaEventDestroy(stop);
      }

      void Start(){cudaEventRecord(start, 0);}

      void Stop(){cudaEventRecord(stop, 0);}

      float Elapsed(){
            float elapsed;
            cudaEventSynchronize(stop);
            cudaEventElapsedTime(&elapsed, start, stop);
            return elapsed;
      }
};

#endif  /* __GPU_TIMER_H__ */

Overwriting GpuTimer.h


In [ ]:
!nvcc matmul.cu -o matmul

In [ ]:
!./matmul

Size of matrix A: 960 640
Size of matrix B: 640 800
Time to allocate memory on the device is: 0.355456 msecs.
Time to copy the Matrix from host to device is: 2.099616 msecs.
Implemented CUDA code ran in: 2.812192 msecs.
Time to copy the resulting Matrix from device to host is: 2.100320 msecs.
Implemented CPU code ran in: 2316.234000 msecs.
Done

In [ ]:
!./matmul 16

Size of matrix A: 960 640
Size of matrix B: 640 800
Time to allocate memory on the device is: 0.297056 msecs.
Time to copy the Matrix from host to device is: 1.110496 msecs.
Implemented CUDA code ran in: 1.792288 msecs.
Time to copy the resulting Matrix from device to host is: 2.076576 msecs.
Implemented CPU code ran in: 2354.113000 msecs.


In [ ]:
!./matmul 32


Size of matrix A: 960 640
Size of matrix B: 640 800
Time to allocate memory on the device is: 0.286240 msecs.
Time to copy the Matrix from host to device is: 1.151712 msecs.
Implemented CUDA code ran in: 1.781952 msecs.
Time to copy the resulting Matrix from device to host is: 2.130816 msecs.
Implemented CPU code ran in: 2346.194000 msecs.


In [ ]:
!nvcc matmulTiled.cu -o matmulTiled

In [ ]:
!./matmulTiled 16

Size of matrix A: 960 640
Size of matrix B: 640 800
Time to allocate memory on the device is: 0.254336 msecs.
Time to copy the Matrix from host to device is: 1.120736 msecs.
Implemented CUDA code ran in: 1.786208 msecs.
Time to copy the resulting Matrix from device to host is: 2.080736 msecs.
Implemented CPU code ran in: 2358.828000 msecs.


In [ ]:
!./matmulTiled 32

Size of matrix A: 960 640
Size of matrix B: 640 800
Time to allocate memory on the device is: 0.370944 msecs.
Time to copy the Matrix from host to device is: 1.135648 msecs.
Implemented CUDA code ran in: 1.781856 msecs.
Time to copy the resulting Matrix from device to host is: 2.654464 msecs.
Implemented CPU code ran in: 2812.543000 msecs.
